<a href="https://colab.research.google.com/github/Ronny-Saputra/Machine-Learning/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tahapan awal : Pembuatan Model dengan Random forest**

### Pengumpulan data & Import Library

Dalam tahapan awal pastikan memasukan library yang dibutuhkan serta tempat untuk mengupload dataset

In [167]:
import pandas as pd
import numpy as np
import sklearn.tree as tree
import sklearn.model_selection as ms
import sklearn.ensemble as ens
import sklearn.metrics as met
import io
import sklearn.preprocessing as prep

In [168]:
from google.colab import files
uploaded = files.upload()

Saving adult.data to adult.data


In [169]:
df = pd.read_csv(io.BytesIO(uploaded['adult.data']), header=None)

### Prapemrosesan Data

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Kita perlu mengganti data kolomnya dengan nama kolom yang sebenarnya sehingga kita akan membuat ini:

In [172]:
data_header = [
    'age','workclass','fnlwgt','education',
    'education-num','marital-status',
    'occupation','relationship','race',
    'sex','capital-gain','capital-loss',
    'hrs-per-week','native','income']

In [173]:
df.columns = data_header
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hrs-per-week,native,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


Sebelum melangkah lebih lanjut, kita perlu memastikan training dataset bebas dari `null`. Kita akan memanggil fungsi `info()` yang akan menampilkan informasi terkait data kita

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hrs-per-week    32561 non-null  int64 
 13  native          32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Dikarenakan data tidak ada `null`, maka tidak perlu dibersihkan

Lalu dicek pada collumn `income` ada berapa saja data yang ada

In [174]:
df['income'].value_counts()

,count
income,
<=50K,24720
>50K,7841


Lalu dikarenakan data masi ada yang dalam bentuk alfanumerik, maka perlu kita encoding agar skicit learn dapat mengelola datanya, berikut merupakan function untuk mengencode isi dari kolom yang masi alphanumerik

In [175]:
def encode_nominal_columns(df):
  le = prep.LabelEncoder()
  for column in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native']:
    df[column + '_code'] = le.fit_transform(df[column])
  return df

df = encode_nominal_columns(df)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,native,income,workclass_code,education_code,marital-status_code,occupation_code,relationship_code,race_code,sex_code,native_code
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,United-States,<=50K,7,9,4,1,1,4,1,39
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,United-States,<=50K,6,9,2,4,0,4,1,39
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,United-States,<=50K,4,11,0,6,1,4,1,39
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,United-States,<=50K,4,1,2,6,0,2,1,39
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,Cuba,<=50K,4,9,2,10,5,2,0,5


dikarenakan kolom yang berisi alfanumerik sudah diencode, maka kita tidak memerlukan collumn yang sudah di encode lagi. maka kita akan menggunakan `.drop()`

In [176]:
X = df.drop(columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native', 'income', 'fnlwgt'], axis=1)
y = df['income']

In [ ]:
X.head()

,age,education-num,capital-gain,capital-loss,hrs-per-week,workclass_code,education_code,marital-status_code,occupation_code,relationship_code,race_code,sex_code,native_code
0,39,13,2174,0,40,7,9,4,1,1,4,1,39
1,50,13,0,0,13,6,9,2,4,0,4,1,39
2,38,9,0,0,40,4,11,0,6,1,4,1,39
3,53,7,0,0,40,4,1,2,6,0,2,1,39
4,28,13,0,0,40,4,9,2,10,5,2,0,5


### Pelatihan model

Melatih model menggunakan Random Forest

In [177]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=0)
model = ens.RandomForestClassifier(n_estimators=100)

Lalu kita training dengan data yang ada menggunakan `.fit()`

In [178]:
model.fit(X_train, y_train)

RandomForestClassifier()

### Pengujian model

In [179]:
y_prediksi = model.predict(X_test)
print('Akurasi = ', met.accuracy_score(y_test, y_prediksi))
print(met.classification_report(y_test, y_prediksi))
print('Skor Kepentingan Fitur = ',model.feature_importances_)

Akurasi =  0.8483033932135728
              precision    recall  f1-score   support

       <=50K       0.88      0.92      0.90      4918
        >50K       0.72      0.63      0.67      1595

    accuracy                           0.85      6513
   macro avg       0.80      0.77      0.79      6513
weighted avg       0.84      0.85      0.84      6513

Skor Kepentingan Fitur =  [0.2161868  0.0972643  0.12573077 0.04015946 0.11083211 0.05082978
 0.03557969 0.07540522 0.08473225 0.11097871 0.0180471  0.0131792
 0.0210746 ]


### Pengujian meningkatkan akurasi model

Untuk meningkatkan akurasi ada beberapa hal yang harus kita lakukan seperti Pemilihan fitur, Jumlah Tree, Kedalaman Tree

Fitur yang digunakan untuk membuat machine learning sangatlah berpengaruh, sehingga dalam pemilihan fitur kita tidak boleh menggunakan semua fitur yang ada, dikarenakan dapat mengurangi akurasi model. kita dapat mengecek fitur apa saja yang dapat berguna untuk pengaruh akurasi machine learning

Disini kita hanya memilih fitur yang memiliki pengaruhnya >= 5% lalu kita akan mencoba menggunakan fitur tersebut

In [180]:
X = df.drop(columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native', 'income', 'fnlwgt','race_code',	'sex_code'	,'native_code','workclass_code', 'education_code'], axis=1)
y = df['income']

Berikut merupakan fitur yang kita gunakan

In [181]:
print(X)

       age  education-num  capital-gain  capital-loss  hrs-per-week  \
0       39             13          2174             0            40   
1       50             13             0             0            13   
2       38              9             0             0            40   
3       53              7             0             0            40   
4       28             13             0             0            40   
...    ...            ...           ...           ...           ...   
32556   27             12             0             0            38   
32557   40              9             0             0            40   
32558   58              9             0             0            40   
32559   22              9             0             0            20   
32560   52              9         15024             0            40   

       marital-status_code  occupation_code  relationship_code  
0                        4                1                  1  
1                

Jumlah tree dan juga kedalaman tree turut membantu dalam meningkatkan akurasi, maka disini kita akan mencoba mengubah untuk meningkatkan akurasi machine learning kita

In [182]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=42)
model1 = ens.RandomForestClassifier(n_estimators=300, max_depth=15, criterion='entropy', bootstrap=True)
model1.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=300)

In [183]:
print('Skor Kepentingan Fitur = ',model1.feature_importances_)

Skor Kepentingan Fitur =  [0.14738466 0.15425488 0.1709262  0.05631601 0.09333368 0.14891627
 0.07105509 0.15781321]


Lalu kita akan menguji akurasi nya

In [184]:
y_prediksi = model1.predict(X_test)
print('Akurasi = ', met.accuracy_score(y_test, y_prediksi))
print(met.classification_report(y_test, y_prediksi))

Akurasi =  0.8679563949025026
              precision    recall  f1-score   support

       <=50K       0.89      0.95      0.92      4942
        >50K       0.79      0.61      0.69      1571

    accuracy                           0.87      6513
   macro avg       0.84      0.78      0.80      6513
weighted avg       0.86      0.87      0.86      6513



### **KESIMPULAN**


1. Kedalaman Tree serta dan Jumlah tree yang digunakan dapat mempengaruhi akurasi machine learning
2. Pemilihan fitur sangartlah penting karena dapat mempengaruhi akurasi machine learning, dimana jika fitur yang tidak berguna dimasukan maka dapat menurunkan akurasi machine learning